# Asymmetric Peak Analysis &mdash; FTIR Experiment

Author: Garrek Stemo \
Date created: August 31, 2020 \
Date updated: October 22, 2020 \
Affiliation: Nara Institute of Science and Technology

This is an interactive notebook for analyzing relatively large sets of spectral data, specifically tuned for angle-resolved FTIR Fabry-Perot spectra. I initially created a command-line tool to do curve fitting and spectral analysis, but this was cumbersome because of the variability from data set to data set and the need to carefully truncate and set bounds on the data. Interactivity also makes it easier to try different fitting models. The notebook uses the lmfit package, which is a wrapper for SciPy's optimize method. It is much more user-friendly and customizable, making it well-adapted to an interactive programming scheme like this one. Lmfit comes with almost all of the fitting functions one might need to perform analysis, but I have included additional functions in a custom `pmath.py` module. The most important function here is the asymmetric Gaussian, Lorentzian, and Voigt functions. Asymmetric broadening of spectra occurs in a wide range of materials, including crystalline solids, nanoparticles, molecular solids, and liquids. The scheme used here to model asymmetries is described in [Korepanov and Sedlovets, 2018](https://arxiv.org/abs/1804.06083). I also wrote a bunch of functions to pull data from FTIR experiments and transfer matrix simulations in the `polariton_processing.py` module. These functions assume the files are named a certain way and are in a particular format, so check out that module for more detail or you can write your own.

## Setup

We use the matplotlib widgets framework to generate interactive plots. Make sure you have the appropriate dependencies installed.

In [1]:
import matplotlib.pyplot as plt
import ipywidgets as widgets
import numpy as np
from scipy import signal
from scipy import optimize
import time
import lmfit as lm
import polariton_processing as pp
import pmath

%matplotlib widget

### Load Data

Assign 'data_directory' the path to a directory containing your angle-resolved data. Each .csv data file must contain "deg##.##_" where the "##.##" is an angle, which could be an integer or not. The underscore is necessary, since the program uses this to extract the angle information from the file name. Then specify an output directory where you would like output data to go.

In [74]:
dppa_cav = ''

angle_data, absorbance_data = pp.get_angle_data_from_dir(dppa_cav)
for d in angle_data:
    print(d[0])

-20.0
-16.0
-12.0
-8.0
-4.0
0.0
4.0
8.0
12.0
16.0
20.0
22.0


## First Data Set

### Visualization

The spectra, either simulated or experimental, probably spans a large domain of frequencies. You can visualize the spectrum from whichever angle you want and pan / zoom around. After zooming around your data, you can truncate each data set to isolate the peaks you want to fit. This is a crucial step, since if there are other peaks in your data that you don't care about the fitting procedure will be disasterous.

In [1]:
# Choose which data point and subrange to examine
lower_bound = 100
upper_bound = 8000
spectrum = angle_data[0]
angle, wavenumber_O, transmittance_O = spectrum
mask = (wavenumber_O >= lower_bound) & (wavenumber_O <= upper_bound)

# Plot the selected spectrum or a subrange
fig1, ax = plt.subplots()

ax.plot(wavenumber_O[mask], transmittance_O[mask])
# ax.set_xlim(lower_bound, upper_bound)

ax.set_title("Spectrum subrange for {} degrees".format(np.round(angle,2)))
ax.set_xlabel(r'Wavenumber (cm$^{-1}$)')
ax.set_ylabel('Transmittance %')
plt.show()

NameError: name 'angle_data' is not defined

In [50]:
plt.close('all')

### Define the Model and Parameters, Check Initial Fit

Now test the fitting functions available in pmath.py to find the one you would like to use. The lmfit package has lots of built-in functions. You can use these, fitting functions in pmath, or your own. Perform the initial fit for this one data set and plot the results before fitting all data sets. You can decide if you need to adjust your initial guess or the model.

Set the model, make initial guesses for the fit parameters, and apply constraints.

In [77]:
omega = wavenumber_O[mask]
y = transmittance_O[mask]

model = lm.models.LorentzianModel(prefix='l1_') + lm.models.LorentzianModel(prefix='l2_')
result = model.fit(y, x=omega, l1_amplitude=1.0, l1_center=1000, l1_sigma=10,
                               l2_amplitude=1.0, l2_center=1000, l2_sigma=10)

print('parameter names: {}'.format(model.param_names))
print('independent variables: {}'.format(model.independent_vars))

parameter names: ['l1_amplitude', 'l1_center', 'l1_sigma', 'l2_amplitude', 'l2_center', 'l2_sigma']
independent variables: ['x']


In [78]:
print(result.params.pretty_print())
result.plot()
plt.show()

Name             Value      Min      Max   Stderr     Vary     Expr Brute_Step
l1_amplitude     1.282     -inf      inf    0.159     True     None     None
l1_center         2152     -inf      inf   0.5347     True     None     None
l1_fwhm          8.775     -inf      inf    1.528    False 2.0000000*l1_sigma     None
l1_height      0.09303     -inf      inf  0.01134    False 0.3183099*l1_amplitude/max(2.220446049250313e-16, l1_sigma)     None
l1_sigma         4.387        0      inf   0.7638     True     None     None
l2_amplitude     43.41     -inf      inf   0.3239     True     None     None
l2_center         2241     -inf      inf   0.1332     True     None     None
l2_fwhm          36.37     -inf      inf   0.3835    False 2.0000000*l2_sigma     None
l2_height         0.76     -inf      inf 0.005577    False 0.3183099*l2_amplitude/max(2.220446049250313e-16, l2_sigma)     None
l2_sigma         18.18        0      inf   0.1917     True     None     None
None


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [79]:
plt.close('all')

## Automatically Fit Peaks From Multiple Data Sets

Now that we have a good fit for the first peak, go through and fit the rest, using the results from this first spectrum as the initial guess for the next set, and so on.

Afterwards, we should inspect the results to make sure the fitting procedure worked. It's not feasible to inspect *every* spectrum for large data sets (I won't stop you!), but you can sample a few and see what they look like by changing the `examine_spectrum` index and generating a plot that will show the actual data (blue dots), the best fit (red), and the initial fit (black dashed line). Since we use the previous best fit for the guess of the next data set, we are essentially plotting the best fit for the current data set alongside the best fit for the previous one.

### Define a fitting procedure

In [80]:
def fit_sets(data_sets, model_func, guess, lbound, ubound):
    """
    Fits all data according to a given model. Model and guess follow lmfit framework.
    """
    guess = guess
    results_list = []
    xy_data = []
    num_sets = len(data_sets)
    for i in range(num_sets):
        angle = data_sets[i][0]
        x_data, y_data = pp.truncate(data_sets[i][1], data_sets[i][2], lbound, ubound)

        result = model_func.fit(y_data, x=x_data,
                                l1_amplitude=guess['l1_amplitude'], l1_center=guess['l1_center'], l1_sigma=guess['l1_sigma'],
                                l2_amplitude=guess['l2_amplitude'], l2_center=guess['l2_center'], l2_sigma=guess['l2_sigma'])

#         print(result.values['w_0'], guess['w_0'])

        results_list.append(result)
        xy_data.append((angle, x_data, y_data))
        guess = result.values
    return results_list, xy_data

### Analyze all angle-resolved data

In [81]:
omega = wavenumber_O[mask]
y = transmittance_O[mask]
sets_to_fit = angle_data #[0:len(angle_data) - 2]
fit_lbound = 1000
fit_ubound = 2000

initial_result = model.fit(y, x=omega, l1_amplitude=1.0, l1_center=1000, l1_sigma=5.0,
                               l2_amplitude=1.0, l2_center=1000, l2_sigma=5.0)

set_results, set_xy = fit_sets(sets_to_fit, model, initial_result.values, fit_lbound, fit_ubound)

### Examine and visualize results

Change `examine_spectrum` to see a specific data set.

In [83]:
examine_spectrum = 0
result = set_results[examine_spectrum]
print(result.params.pretty_print())

angle, wavenumber, transmittance = set_xy[examine_spectrum]
# mask = (wavenumber >= lower_bound) & (wavenumber <= upper_bound)

fig3, ax = plt.subplots()

ax.plot(wavenumber, transmittance, 'bo', markersize=3, label='raw data')
ax.plot(wavenumber, result.init_fit, 'k--', label='initial fit')
ax.plot(wavenumber, result.best_fit, 'r-', label='best fit')

ax.set_title("Spectrum for {} degrees".format(np.round(angle, 1)))
plt.legend()
plt.show()

Name             Value      Min      Max   Stderr     Vary     Expr Brute_Step
l1_amplitude     1.286     -inf      inf   0.1551     True     None     None
l1_center         2152     -inf      inf   0.5208     True     None     None
l1_fwhm          8.803     -inf      inf    1.488    False 2.0000000*l1_sigma     None
l1_height      0.09304     -inf      inf  0.01101    False 0.3183099*l1_amplitude/max(2.220446049250313e-16, l1_sigma)     None
l1_sigma         4.401        0      inf   0.7441     True     None     None
l2_amplitude     43.36     -inf      inf   0.3164     True     None     None
l2_center         2241     -inf      inf   0.1297     True     None     None
l2_fwhm          36.31     -inf      inf   0.3745    False 2.0000000*l2_sigma     None
l2_height       0.7603     -inf      inf 0.005445    False 0.3183099*l2_amplitude/max(2.220446049250313e-16, l2_sigma)     None
l2_sigma         18.15        0      inf   0.1873     True     None     None
None


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [98]:
plt.close('all')

## Dispersion Relation

Now we plot the peak positions from the fitting procedure as a function of cavity angle and fit the polariton eigenenergies to the upper and lower polariton curves.

In [84]:
fig4, ax = plt.subplots()

theta = []
LP = []
UP = []

for i in np.arange(len(set_results)):
    theta.append(set_xy[i][0])
    LP.append(set_results[i].params['l1_center'].value)
    UP.append(set_results[i].params['l2_center'].value)
    
ax.scatter(theta, LP)
ax.scatter(theta, UP)

plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [59]:
plt.close('all')

In [85]:
x0 = [2100, 2170, 110, 1.55]
splitting_fit = pp.splitting_least_squares(x0, theta, LP, UP)

print(splitting_fit.message)
print(splitting_fit.x)

`ftol` termination condition is satisfied.
[2.18712475e+03 2.16220486e+03 6.01014365e+01 1.85918861e+00]


In [86]:
theta_plot = np.linspace(-30, 30, 100)
theta_rad = [np.pi/180*t for t in theta_plot]

E0, Ev, V, n_eff = splitting_fit.x
# Ev = 2171
# E0 = 2159
# n_eff = 1.5

En = pmath.coupled_energies(theta_rad, E0, Ev, V, n_eff, branch=0)
Ep = pmath.coupled_energies(theta_rad, E0, Ev, V, n_eff, branch=1)
E_ph = pmath.cavity_mode_energy(theta_rad, E0, n_eff)
fig10, ax = plt.subplots()

ax.plot(theta_plot, En, 'r-')
ax.plot(theta_plot, Ep, 'r-')
ax.plot(theta_plot, np.full(len(theta_plot), Ev), color='dimgray', linestyle='dashed')
ax.plot(theta_plot, E_ph, color='dimgray', linestyle='dashed')
ax.scatter(theta, LP, color='darkblue', marker='o', s=20)
ax.scatter(theta, UP, color='darkblue', marker='o', s=20)

rabi_text = r'$\Omega_R = %.2f$' % (V)
ax.text(20, 2150, rabi_text, bbox=dict(boxstyle='round, pad=0.5', facecolor='white'))

plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [ ]:
plt.close('all')